In [14]:
%cd ../

/u02/thanh/workplace


In [15]:
import plasma.functional as F

from plasma.meta.object_graph import ContextManager, Manager

In [16]:
class Handler(F.AutoPipe):

    def __init__(self, name, age, test=5):
        super().__init__()
        self.name = name
        self.age = age
        self.test = test

In [17]:
class Flow(F.AutoPipe):

    def __init__(self, handler, bee) -> None:
        super().__init__()
        self.handler = handler

In [18]:
class Service(F.AutoPipe):

    def __init__(self, flow:Flow, handler:Handler, test_factory:dict) -> None:
        super().__init__()
        print(test_factory)
        self.flow = flow
        self.handler = handler

In [19]:
class AnotherHandler(F.AutoPipe):
    
    def __init__(self, test_factory:dict):
        super().__init__()
        
        self.test_factory = test_factory

In [20]:
manager =  ContextManager()
manager

In [21]:
service_manager = manager\
    .context('service_components')\
        .add_dependency('main', Service)\
        .add_dependency('another_handler', AnotherHandler)

service_manager

service_components
  |-> main
    |-> flow: Flow
    |-> handler: Handler
    |-> test_factory: dict
----------------------------------------------------------------------------------------------------
service_components
  |-> another_handler
    |-> test_factory
----------------------------------------------------------------------------------------------------

In [22]:
factory = service_manager.factory('test_factory')
factory['hehe'] = 1

service_manager

service_components
  |-> main
    |-> flow: Flow
    |-> handler: Handler
    |-> test_factory: DependencyFactory
      |-> hehe = int
----------------------------------------------------------------------------------------------------
service_components
  |-> another_handler
    |-> test_factory...
----------------------------------------------------------------------------------------------------

In [23]:
main, *_ = service_manager.inputs('main', 'another_handler')
main

plasma.meta.object_graph.contexts.opened.Required

In [24]:
package_manager = manager\
    .context('package')\
        .add_dependency('flow', Flow)\
        .add_dependency('handler', Handler)
package_manager           

package
  |-> flow
    |-> handler
      |-> name
      |-> age
      |-> test = int
    |-> bee
----------------------------------------------------------------------------------------------------

In [25]:
service_manager.link_name(package_manager)

service_components
  |-> main
    |-> flow: Flow --> package.flow...
    |-> handler: Handler --> package.handler...
    |-> test_factory: DependencyFactory
      |-> hehe = int
----------------------------------------------------------------------------------------------------
service_components
  |-> another_handler
    |-> test_factory...
----------------------------------------------------------------------------------------------------

In [27]:
inputs = service_manager.inputs('main', 'another_handler')
inputs

{
  "package": {
    "name": Required,
    "age": Required,
    "test": 5,
    "bee": Required
  },
  "hehe": 1
}

In [ ]:
main, another_handler = service_manager\
                              .run('main', 'another_handler',
                                   package={
                                        'name': 'test', 
                                        'age': 12, 
                                        'test': 'test', 
                                        'bee': 'buzz buzz'
                                        })

In [ ]:
another_handler